In [3]:
 import numpy as np

# Parameter values
delta = 0.08
alpha = 0.3
beta = 0.96
sigma = 1.01
lambda_val = 0.1
T = 200
A = 1

# Steady state capital stock and initial condition
kss = ((1 / beta - (1 - delta)) / A / alpha) ** (1 / (alpha - 1))
k0 = lambda_val * kss

# Solution for path ksol(t)
ksol = np.zeros(T)
ksol[0] = k0

for t in range(1, T):
    kguess = np.zeros(T)
    kguess[0] = ksol[t - 1]

    # Step 1: Set boundaries for interval that ksol(t) must lie in
    kmin = ksol[t - 1]
    kmax = kss

    # Step 2: Pick a point in the interval as a hypothetical choice for k(t)
    while abs(kmax - kmin) > 0.00000015 * kss:
        kn = 0.5 * (kmin + kmax)
        kguess[1] = kn

        # Step 3: Determine the rest of the path given the choice for k(t)
        stop = 0
        i = 1

        while stop < 1:
            i += 1

            # Implied value for kguess(i)
            rhs = A * kguess[i - 1] ** alpha + (1 - delta) * kguess[i - 1]
            kguess[i] = rhs - (beta * (A * alpha * kguess[i - 1] ** (alpha - 1) + (1 - delta))) ** (1 / sigma) * (
                    rhs - kguess[i - 1])

            # Check to see if kguess is going to zero (i.e., did we enter region I)
            if kguess[i] <= kguess[i - 1]:
                kmin = kn
                stop = 1
            else:
                kguess[i] = kguess[i]

            # Check to see if kguess is going beyond kss (i.e., did we enter region III)
            if kguess[i] > kss:
                kmax = kn
                stop = 1
            else:
                kguess[i] = kguess[i]

    # We have now determined (within the limits of our approximation) the next value of k(t)
    ksol[t] = kguess[1]

# We now know the whole sequence ksol(t). We can also determine the sequences for consumption and utility
c = A * ksol[:-1] ** alpha + (1 - delta) * ksol[:-1] - ksol[1:]
c = np.append(c, c[-1])  # To match MATLAB indexing for c(T)

# Correct for cases where c is negative or zero
c = np.maximum(c, 1e-8)

uv = (c ** (1 - sigma)) / (1 - sigma)
betav = np.zeros(T)
betav[0] = 1
for j in range(1, T):
    betav[j] = beta * betav[j - 1]

utot = np.sum(betav * uv)
srate = 1 - c / (A * ksol ** alpha)

# Print or use the results as needed


C:\Users\tanay\AppData\Local\Temp\ipykernel_44896\1666524754.py:76: RuntimeWarning: divide by zero encountered in divide
  srate = 1 - c / (A * ksol ** alpha)
